# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
item_per_customer = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum })
item_per_customer

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
products_customer = pd.pivot_table(item_per_customer,
                     index='ProductName',
                     columns='CustomerID', 
                     values='Quantity',
                     fill_value=0)

products_customer.T

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
pivot2= products_customer.transpose()
dist_eucl = 1 / (1 + squareform(pdist(pivot2, metric='euclidean')))
dist_eucl
distances = pd.DataFrame(dist_eucl, index=pivot2.index, 
                         columns=pivot2.index)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarities = pd.DataFrame(distances[200].sort_values(ascending=False)[1:])
similarities =similarities.head(5).index

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
pivot = pd.DataFrame(products_customer, columns=similarities)
pivot

CustomerID,3317,1072,3535,1920,3909
ProductName,,,,,
Anchovy Paste - 56 G Tube,0,0,1,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,1,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0
Appetizer - Sausage Rolls,0,1,0,0,0
Apricots - Dried,0,0,1,0,0
...,...,...,...,...,...
Yeast Dry - Fermipan,0,1,0,0,0
Yoghurt Tubes,0,0,1,0,0
"Yogurt - Blueberry, 175 Gr",0,0,0,0,0


In [9]:
item_per_customer.loc[similarities]

Quantity
CustomerID ProductName                             
3317       Bay Leaf                               1
           Beef - Ground Medium                   1
           Beef - Rib Eye Aaa                     1
           Butter - Unsalted                      1
           Cheese - Victor Et Berthold            1
...                                             ...
3909       Wine - Charddonnay Errazuriz           2
           Wine - Fume Blanc Fetzer               1
           Wine - Magnotta, Merlot Sr Vqa         1
           Wine - Toasted Head                    1
           Wine - Valpolicella Masi               1

[262 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
step5 = item_per_customer.loc[similarities]
step6 = step5.groupby('ProductName').agg({'Quantity': sum}).sort_values('Quantity', ascending=False)
step6

,Quantity
ProductName,
Soup - Campbells Bean Medley,4
Muffin - Carrot Individual Wrap,3
Bay Leaf,3
Pork - Kidney,3
"Pepper - Black, Whole",3
...,...
Kiwi,1
Knife Plastic - White,1
Lambcasing,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
prod_200 = products_customer[200]
prod_200

step7 = pd.merge(left=step6, right=prod_200, on='ProductName' )
step7[step7[200]==0].head()

,Quantity,200
ProductName,,
Soup - Campbells Bean Medley,4,0
Muffin - Carrot Individual Wrap,3,0
Bay Leaf,3,0
Pork - Kidney,3,0
Wanton Wrap,3,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
customers = data['CustomerID'].unique()


In [13]:

recomm_dict = dict()

recomend = {}
def recommendations(customer_id):
    dist_eucl = 1 / (1+ squareform(pdist(
products_customer.T, metric='euclidean')))
    distances = pd.DataFrame(dist_eucl, index=
products_customer.T.index, 
                         columns=
products_customer.T.index)
    similarities = list(distances[customer_id].sort_values(ascending=False)[1:6].index)
    qty=item_per_customer.loc[similarities]
    qty_simil=qty.groupby('ProductName').sum().sort_values('Quantity', ascending=False)
    prod_200 = products_customer
    merge = qty_simil.merge(prod_200, on='ProductName')
    recommendation=list(merge[merge[customer_id] == 0].sort_values('Quantity', ascending=False).head(5).index)
    recomm_dict[customer_id] = recommendation 
    return recomend 

for id_ in customers:
    recommendations(id_)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [18]:
pd.DataFrame.from_dict(recomm_dict).T

,0,1,2,3,4
61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
23548,Squid - Tubes / Tenticles 10/20,Wanton Wrap,Puree - Mocha,Pernod,Sprouts - Baby Pea Tendrils
78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
...,...,...,...,...,...
49005,"Pasta - Penne, Rigate, Dry",Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab"
41286,Veal - Inside,Wonton Wrappers,"Veal - Inside, Choice",Bread Crumbs - Japanese Style,Wasabi Powder
85878,Olives - Kalamata,Wine - White Cab Sauv.on,Veal - Eye Of Round,Bread - Calabrese Baguette,Cheese - Wine
68506,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,"Rosemary - Primerba, Paste",Scallops - 10/20,"Oregano - Dry, Rubbed"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [17]:
recommend_2 = {}

def recommendations2(customer_id):
    dist_manhattan = 1 / (1+ squareform(pdist(
products_customer.T, metric='cityblock')))
    distances = pd.DataFrame(dist_manhattan, index=
products_customer.T.index, 
                         columns=
products_customer.T.index)
    similarities = list(distances[customer_id].sort_values(ascending=False)[1:6].index)
    qty=item_per_customer.loc[similarities]
    qty_simil=qty.groupby('ProductName').sum().sort_values('Quantity', ascending=False)
    prod_200 = products_customer
    merge = qty_simil.merge(prod_200, on='ProductName')
    recommendation=list(merge[merge[customer_id] == 0].sort_values('Quantity', ascending=False).head(5).index)
    recommend_2[customer_id] = recommendation 
    return recommend_2 

for id_ in customers:
    recommendations(id_)

In [16]:
recommend_2 